## Funciones de utilidad para calcular la matriz de costos

In [11]:
import math
import requests

def haversine(coord1, coord2):
    R = 6371  # Radio de la Tierra en kilómetros
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    d_phi = math.radians(lat2 - lat1)
    d_lambda = math.radians(lon2 - lon1)
    a = math.sin(d_phi / 2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(d_lambda / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c  # Distancia en kilómetros

def compute_distance_matrix(data):
    nodes = data['N']
    node_coords = [data['UbicacionNodo'][i] for i in nodes]
    
    # Prepare coordinates for the OSRM API in lon,lat format
    coords_str = ';'.join([f"{lon},{lat}" for lat, lon in node_coords])
    
    # OSRM distance computation for all ground vehicles (single call)
    ground_vehicle_types = [cv for cv in data['CV'] if cv != 'Aereo']
    url = f"https://router.project-osrm.org/table/v1/driving/{coords_str}"
    print(url)
    params = {
        'annotations': 'distance'
    }
    try:
        # Single API call
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an HTTPError if status != 200
        result = response.json()
        if 'distances' not in result:
            raise ValueError("No 'distances' in OSRM response")
        
        distances = result['distances']
        for tv in ground_vehicle_types:
            for i, from_node in enumerate(nodes):
                for j, to_node in enumerate(nodes):
                    distance = distances[i][j] / 1000  # Convert meters to kilometers
                    data['d'][tv][from_node][to_node] = distance

    except Exception as e:
        print(f"Error fetching distances for ground vehicles: {e}")
        for tv in ground_vehicle_types:
            for i in nodes:
                for j in nodes:
                    data['d'][tv][i][j] = float('inf')  # Default to a large number in case of failure
    
    # Calculate distances for drones using Haversine formula
    if 'Dron' in data['TV']:
        for i in nodes:
            for j in nodes:
                coord_i = data['UbicacionNodo'][i]
                coord_j = data['UbicacionNodo'][j]
                distance = haversine(coord_i, coord_j)
                data['d']['Aereo'][i][j] = distance

## Función para resolver el modelo de optimización

In [12]:
def solve_vehicle_routing_problem(data):
    from pyomo.environ import ConcreteModel, Var, Objective, SolverFactory, NonNegativeReals, Binary, Set, Param, Constraint, minimize, Any

    model = ConcreteModel()

    # Sets
    model.N = Set(initialize=data["N"])
    model.V = Set(initialize=data["V"])
    model.TV = Set(initialize=data["TV"])
    model.TP = Set(initialize=data["TP"])
    model.CV = Set(initialize=data["CV"])

    # Parameters
    model.CostoCarga = Param(initialize=data["CostoCarga"])
    model.CostoDescarga = Param(initialize=data["CostoDescarga"])
    model.VelocidadCarga = Param(initialize=data["VelocidadCarga"])
    model.VelocidadDescarga = Param(initialize=data["VelocidadDescarga"])

    model.TipoNodo = Param(model.N, initialize=data["TipoNodo"])
    model.DemandaEnvio = Param(model.N, initialize=data["DemandaEnvio"])
    model.CapacidadDeposito = Param(model.N, initialize=data["CapacidadDeposito"])

    model.TipoVehiculo = Param(model.V, initialize=data["TipoVehiculo"])
    model.CapacidadVehiculo = Param(model.V, initialize=data["CapacidadVehiculo"])
    model.RangoVehiculo = Param(model.V, initialize=data["RangoVehiculo"])

    model.TarifaFleteTipoVehiculo = Param(model.TV, initialize=data["TarifaFleteTipoVehiculo"])
    model.TarifaHorariaTipoVehiculo = Param(model.TV, initialize=data["TarifaHorariaTipoVehiculo"])
    model.MantenimientoTipoVehiculo = Param(model.TV, initialize=data["MantenimientoTipoVehiculo"])
    model.CostoRecargaTipoVehiculo = Param(model.TV, initialize=data["CostoRecargaTipoVehiculo"])
    model.VelocidadTipoVehiculo = Param(model.TV, initialize=data["VelocidadTipoVehiculo"])
    model.EficienciaTipoVehiculo = Param(model.TV, initialize=data["EficienciaTipoVehiculo"])

    model.d = Param(model.CV, model.N, model.N, initialize=lambda model, cv, i, j: data["d"][cv][i][j])

    # Parámetro auxiliar que soporta la jerarquía que ayuda a disminuir el tamaño de la matriz de costos
    model.CategoriaTipoVehiculo = Param(model.TV, initialize=data["CategoriaTipoVehiculo"], within=Any)

    # Sets auxiliares
    model.Clients = Set(initialize=[c for c in model.N if model.TipoNodo[c] == 'Cliente'])
    model.Deposits = Set(initialize=[d for d in model.N if model.TipoNodo[d] == 'Depósito'])

    # Definir el número total de clientes
    model.n_c = Param(initialize=len(model.Clients))

    # Decision Variables
    model.z = Var(model.Clients, model.V, domain=Binary)
    model.r = Var(model.N, model.N, model.V, domain=Binary)
    model.a = Var(model.V, model.Deposits, domain=Binary)
    model.u = Var(model.N, model.V, domain=NonNegativeReals)
    # Auxiliary variable for linearization of the 'no more than one route per vehicle' restrictions
    model.y = Var(model.Clients, model.V, model.Deposits, domain=Binary)

    # Variables para cada componente de costo
    model.CostCarTot = Var(domain=NonNegativeReals)
    model.CostDistTot = Var(domain=NonNegativeReals)
    model.CostTpTot = Var(domain=NonNegativeReals)
    model.CostRecTot = Var(domain=NonNegativeReals)
    model.CostMant = Var(domain=NonNegativeReals)

    # Define the TotalCost variable
    model.TotalCost = Var(domain=NonNegativeReals)

    # Define las restricciones para cada componente de costo

    # CostCarTot Constraint
    def CostCarTot_rule(model):
        loading_unloading_cost = model.CostoCarga * model.VelocidadCarga + model.CostoDescarga * model.VelocidadDescarga
        total_demand = sum(model.DemandaEnvio[c] * model.z[c, v] for v in model.V for c in model.Clients)
        return model.CostCarTot == loading_unloading_cost * total_demand

    model.CostCarTotConstraint = Constraint(rule=CostCarTot_rule)

    # CostDistTot Constraint
    def CostDistTot_rule(model):
        return model.CostDistTot == sum(
            model.TarifaFleteTipoVehiculo[model.TipoVehiculo[v]] * sum(
                model.d[model.CategoriaTipoVehiculo[model.TipoVehiculo[v]], i, j] * model.r[i, j, v]
                for i in model.N for j in model.N
            )
            for v in model.V
        )

    model.CostDistTotConstraint = Constraint(rule=CostDistTot_rule)

    # CostTpTot Constraint
    def CostTpTot_rule(model):
        return model.CostTpTot == sum(
            model.TarifaHorariaTipoVehiculo[model.TipoVehiculo[v]] * sum(
                model.d[model.CategoriaTipoVehiculo[model.TipoVehiculo[v]], i, j] * model.r[i, j, v]
                for i in model.N for j in model.N
            ) / model.VelocidadTipoVehiculo[model.TipoVehiculo[v]]
            for v in model.V
        )

    model.CostTpTotConstraint = Constraint(rule=CostTpTot_rule)

    # CostRecTot Constraint
    def CostRecTot_rule(model):
        return model.CostRecTot == sum(
            model.CostoRecargaTipoVehiculo[model.TipoVehiculo[v]] * sum(
                model.d[model.CategoriaTipoVehiculo[model.TipoVehiculo[v]], i, j] * model.r[i, j, v]
                for i in model.N for j in model.N
            ) / model.EficienciaTipoVehiculo[model.TipoVehiculo[v]]
            for v in model.V
        )

    model.CostRecTotConstraint = Constraint(rule=CostRecTot_rule)

    # CostMant Constraint
    def CostMant_rule(model):
        return model.CostMant == sum(
            model.MantenimientoTipoVehiculo[model.TipoVehiculo[v]] for v in model.V
        )

    model.CostMantConstraint = Constraint(rule=CostMant_rule)

    # TotalCost Constraint
    def TotalCost_rule(model):
        return model.TotalCost == model.CostCarTot + model.CostDistTot + model.CostTpTot + model.CostRecTot + model.CostMant

    model.TotalCostConstraint = Constraint(rule=TotalCost_rule)

    # Definir la Función Objetivo
    model.obj = Objective(expr=model.TotalCost, sense=minimize)

    # Restricciones

    # Restricción 1: conservación de flujo
    def flow_conservation_rule(model, v, j):
        return sum(model.r[i, j, v] for i in model.N) <= sum(model.r[j, k, v] for k in model.N)

    model.FlowConservation = Constraint(model.V, model.N, rule=flow_conservation_rule)

    # Restricción 2: limitación de entradas y salidas
    def entry_limit_rule(model, v, j):
        return sum(model.r[i, j, v] for i in model.N) <= 1

    def exit_limit_rule(model, v, j):
        return sum(model.r[j, k, v] for k in model.N) <= 1

    model.EntryLimit = Constraint(model.V, model.N, rule=entry_limit_rule)
    model.ExitLimit = Constraint(model.V, model.N, rule=exit_limit_rule)

    # Restricciones para restricción 4: solo se debe asignar un depósito a un vehículo con clientes asignados, y viceversa
    # solo se puede asignar como mucho un depósito a un vehículo
    def max_one_deposit_assigned(model, v):
        return sum(model.a[v, d] for d in model.Deposits) <= 1

    model.DepositAssignment1 = Constraint(model.V, rule=max_one_deposit_assigned)

    # Linearized Constraint: Upper bound
    def deposit_assignment_upper_rule(model, v):
        return sum(model.z[c, v] for c in model.Clients) <= model.n_c * sum(model.a[v, d] for d in model.Deposits)

    model.DepositAssignmentUpper = Constraint(model.V, rule=deposit_assignment_upper_rule)

    # Linearized Constraint: Lower bound
    def deposit_assignment_lower_rule(model, v):
        return sum(model.z[c, v] for c in model.Clients) >= sum(model.a[v, d] for d in model.Deposits)

    model.DepositAssignmentLower = Constraint(model.V, rule=deposit_assignment_lower_rule)

    # Restricción 6: solo salida desde el depósito asignado
    def departure_from_assigned_depot_rule(model, v, d):
        return sum(model.r[d, i, v] for i in model.N) == model.a[v, d]

    model.DepartureFromAssignedDepot = Constraint(model.V, model.Deposits, rule=departure_from_assigned_depot_rule)

    # Restricción 7: solo regreso al depósito asignado
    def return_to_assigned_depot_rule(model, v, d):
        return sum(model.r[i, d, v] for i in model.N) == model.a[v, d]

    model.ReturnToAssignedDepot = Constraint(model.V, model.Deposits, rule=return_to_assigned_depot_rule)

    # Restricción 8: para asignación de clientes a rutas
    def client_assignment_rule(model, v, c):
        return model.z[c, v] <= sum(model.r[j, c, v] for j in model.N)

    model.ClientAssignment = Constraint(model.V, model.Clients, rule=client_assignment_rule)

    # Restricción 9: para evitar que los vehículos recorran nodos para los cuales no tienen clientes asignados
    def client_route_assignment_rule(model, v):
        return sum(model.r[i, j, v] for i in model.N for j in model.N) - 1 <= sum(
            model.z[c, v] for c in model.Clients
        )

    model.ClientRouteAssignment = Constraint(model.V, rule=client_route_assignment_rule)

    # Restricción 11: cada cliente es atendido una vez
    def each_client_attended_once_rule(model, c):
        return sum(model.z[c, v] for v in model.V) == 1

    model.EveryClientServedOnce = Constraint(model.Clients, rule=each_client_attended_once_rule)

    # Restricción 12: para capacidad de carga del vehículo
    def vehicle_capacity_rule(model, v):
        return sum(model.z[c, v] * model.DemandaEnvio[c] for c in model.Clients) <= model.CapacidadVehiculo[v]

    model.VehicleCapacity = Constraint(model.V, rule=vehicle_capacity_rule)

    # Linking constraints for y[i, v, d]
    def linking_constraint_1(model, c, v, d):
        return model.y[c, v, d] <= model.a[v, d]

    def linking_constraint_2(model, c, v, d):
        return model.y[c, v, d] <= model.z[c, v]

    def linking_constraint_3(model, c, v, d):
        return model.y[c, v, d] >= model.a[v, d] + model.z[c, v] - 1

    model.LinkingConstraint1 = Constraint(model.Clients, model.V, model.Deposits, rule=linking_constraint_1)
    model.LinkingConstraint2 = Constraint(model.Clients, model.V, model.Deposits, rule=linking_constraint_2)
    model.LinkingConstraint3 = Constraint(model.Clients, model.V, model.Deposits, rule=linking_constraint_3)

    # Restricción 13: para capacidad del depósito
    def depot_storage_capacity_rule(model, d):
        return sum(model.y[i, v, d] * model.DemandaEnvio[i] for i in model.Clients for v in model.V) <= model.CapacidadDeposito[d]

    model.DepotStorageCapacity = Constraint(model.Deposits, rule=depot_storage_capacity_rule)

    # Restricciones 14 MTZ para eliminación de subciclos
    def mtz_rule(model, v, i, j):
        return model.u[i, v] - model.u[j, v] + model.n_c * model.r[i, j, v] <= model.n_c - 1

    model.MTZ = Constraint(model.V, model.Clients, model.Clients, rule=mtz_rule)

    # Restricción 15 para Asignación de posición en la ruta para depósitos
    def assign_position_depot_rule(model, v, d):
        return model.u[d, v] == 0

    model.AssignPositionDepot = Constraint(model.V, model.Deposits, rule=assign_position_depot_rule)

    # Restricción 16 para Rango de posiciones en la ruta para clientes
    def route_position_range_rule(model, v, i):
        return (1, model.u[i, v], model.n_c)

    model.RoutePositionRange = Constraint(model.V, model.Clients, rule=route_position_range_rule)

    # Resolver el modelo
    solver_name = 'gurobi'
    solver = SolverFactory(solver_name)

    if solver_name == 'gurobi':
         # Establecer un límite de tiempo (en segundos)
        #solver.options['TimeLimit'] = 60 * 60  # 30 min
        
        # Establecer una brecha de optimalidad aceptable (por ejemplo, 5%)
        #solver.options['MIPGap'] = 0.1  # 1% de brecha
        
        # Incrementar el énfasis en heurísticas para encontrar buenas soluciones rápidamente
        #solver.options['Heuristics'] = 0.7  # Aumenta el esfuerzo heurístico (valor por defecto es 0.05)
                
                
        # Enfatizar la búsqueda de soluciones factibles
        #solver.options['MIPFocus'] = 1  # Prioriza encontrar soluciones factibles
        
        # Ajustar la cantidad de hilos para procesamiento paralelo
        #solver.options['Threads'] = 4  # Ajusta según los núcleos de tu CPU
        
        # Limitar la cantidad de nodos explorados (opcional)
        # solver.options['NodeLimit'] = 100000  # Por ejemplo, 100,000 nodos
        

        # Establecer una semilla para reproducibilidad (opcional)
        # solver.options['Seed'] = 12345
        
        # Activar cortes adicionales para mejorar la eficiencia (opcional)
        #solver.options['Cuts'] = 3  # Nivel de cortes (0: ninguno, 3: alto)
        
        # Desactivar algunas heurísticas internas menos efectivas (opcional)
        # solver.options['Heuristics'] = 0.0  # Desactiva heurísticas si es necesario
        pass
    elif solver_name == 'cbc':
        # Configuración de opciones para CBC
        solver.options['seconds'] = 60*30  # En segundos
        solver.options['ratioGap'] = 0.05   # Gap de optimalidad aceptable del 5%
        solver.options['heuristics'] = 0.5   # Incrementar búsqueda heurística
        solver.options['threads'] = 4        # Número de hilos
        # Otras opciones pueden ser configuradas según necesidad

    results = solver.solve(model, tee=True)

    # Extraer la solución
    solution = {
        'TotalCost': model.TotalCost.value,
        'CostCarTot': model.CostCarTot.value,
        'CostDistTot': model.CostDistTot.value,
        'CostTpTot': model.CostTpTot.value,
        'CostRecTot': model.CostRecTot.value,
        'CostMant': model.CostMant.value,
        'z': {(c, v): (0 if model.z[c, v].value is None else int(model.z[c, v].value)) for c in model.Clients for v in model.V},
        'r': {(i, j, v): (0 if model.r[i, j, v].value is None else int(model.r[i, j, v].value)) for i in model.N for j in model.N for v in model.V},
        'a': {(v, d): (0 if model.a[v, d].value is None else int(model.a[v, d].value)) for v in model.V for d in model.Deposits},
        'u': {(i, v): (0.0 if model.u[i, v].value is None else float(model.u[i, v].value)) for i in model.N for v in model.V}
    }

    solution_2 = {
        'TotalCost': model.TotalCost.value,
        'CostCarTot': model.CostCarTot.value,
        'CostDistTot': model.CostDistTot.value,
        'CostTpTot': model.CostTpTot.value,
        'CostRecTot': model.CostRecTot.value,
        'CostMant': model.CostMant.value,
        'z': {str((c, v)): (0 if model.z[c, v].value is None else int(model.z[c, v].value)) for c in model.Clients for v in model.V},
        'r': {str((i, j, v)): (0 if model.r[i, j, v].value is None else int(model.r[i, j, v].value)) for i in model.N for j in model.N for v in model.V},
        'a': {str((v, d)): (0 if model.a[v, d].value is None else int(model.a[v, d].value)) for v in model.V for d in model.Deposits},
        'u': {str((i, v)): (0.0 if model.u[i, v].value is None else float(model.u[i, v].value)) for i in model.N for v in model.V}
    }

    return solution, solution_2


## Función para visualizar rutas encontradas

In [13]:
def print_vehicle_routes(solution, data):
    N = data['N']
    V = data['V']
    TipoNodo = data['TipoNodo']
    TipoVehiculo = data['TipoVehiculo']
    
    for v in V:
        print(f"Vehículo {v} ({TipoVehiculo[v]}):")
        # Obtener los arcos utilizados por el vehículo v
        arcs = {(i, j) for (i, j, vp) in solution['r'] if vp == v and solution['r'][(i, j, v)] > 0.5}
        if not arcs:
            print("  No hay ruta asignada.")
            continue
        
        # Encontrar el depósito asignado al vehículo
        depots = [d for (vp, d) in solution['a'] if vp == v and solution['a'][(v, d)] > 0.5 and TipoNodo[d] == 'Depósito']
        if not depots:
            print("  Error: No se encontró depósito asignado.")
            continue
        start_node = depots[0]
        
        # Construir la ruta a partir del depósito
        route = [start_node]
        current_node = start_node
        visited = {start_node}
        while True:
            # Encontrar el siguiente nodo desde el nodo actual
            next_nodes = [j for (i, j) in arcs if i == current_node]
            if len(next_nodes) == 0:
                print("  Error: Ruta incompleta, no hay salida desde el nodo", current_node)
                break
            elif len(next_nodes) > 1:
                print("  Error: Múltiples salidas desde el nodo", current_node)
                break
            next_node = next_nodes[0]
            route.append(next_node)
            if next_node == start_node:
                print("  Ruta (circular):", ' -> '.join(str(n) for n in route))
                break
            if next_node in visited:
                print("  Error: Se detectó un ciclo en la ruta que no regresa al depósito.")
                break
            visited.add(next_node)
            current_node = next_node
    print(f"Costo Total: {solution['TotalCost']}")

## Funciones de utilidad para cargar los datos

In [14]:
import pandas as pd
import os

def is_float_with_zero_decimals(s):
    try:
        num = float(s)
        return num.is_integer()
    except ValueError:
        return False

def select_base_case():
    print("Select a base case (1-4):")
    base_case = input("Enter base case number: ")
    if base_case not in ['0','1', '2', '3', '4']:
        raise ValueError("Invalid base case number.")
    os.environ['CASE_NUMBER'] = base_case
    return base_case

def get_base_case_paths(base_case):
    # Predefined paths for base cases
    base_case_paths = {
        '0': {
            'clients': ['case_1_base_viejo/Clients.csv'],
            'vehicles': ['case_1_base_viejo/multi_vehicles.csv'],
            'depots': ['multi_depots.csv'],
            'depot_capacities': [],
            'loading_products': ['loading_costs.csv'],
            'vehicle_types': ['vehicles_data.csv']
        },
        '1': {
            'clients': ['case_1_base/Clients.csv'],
            'vehicles': ['case_1_base/Vehicles.csv'],
            'depots': ['case_1_base/Depots.csv'],
            'depot_capacities': [],
            'loading_products': ['loading_costs.csv'],
            'vehicle_types': ['vehicles_data.csv']
        },
        '2': {
            'clients': ['case_2_cost/Clients.csv'],
            'vehicles': ['case_2_cost/Vehicles.csv'],
            'depots': ['case_2_cost/Depots.csv'],
            'depot_capacities': [],
            'loading_products': ['loading_costs.csv'],
            'vehicle_types': ['vehicles_data.csv']
        },
        '3': {
            'clients': ['case_3_supply_limits/Clients.csv'],
            'vehicles': ['case_3_supply_limits/Vehicles.csv'],
            'depots': ['case_3_supply_limits/Depots.csv'],
            'depot_capacities': ['case_3_supply_limits/DepotCapacities.csv'],
            'loading_products': ['loading_costs.csv'],
            'vehicle_types': ['vehicles_data.csv']
        },
        '4': {
            'clients': ['case_4_multi_product/Clients.csv'],
            'vehicles': ['case_4_multi_product/Vehicles.csv'],
            'depots': ['case_4_multi_product/Depots.csv'],
            'depot_capacities': ['case_4_multi_product/DepotCapacities.csv'],
            'loading_products': ['loading_costs.csv'],
            'vehicle_types': ['vehicles_data.csv']
        },
        # Add paths for other base cases
    }
    return base_case_paths[base_case]

def get_manual_paths():
    paths = {}
    paths['clients'] = input_paths("clients")
    paths['vehicles'] = input_paths("vehicles")
    paths['depots'] = input_paths("depots")
    paths['depot_capacities'] = input_paths("depot capacities (press Enter to skip)", optional=True)
    paths['loading_products'] = input_paths("loading products")
    paths['vehicle_types'] = input_paths("vehicle types")
    return paths

def input_paths(csv_type, optional=False):
    paths = input(f"Enter paths to {csv_type} CSV files, separated by commas: ").split(',')
    paths = [path.strip() for path in paths if path.strip()]
    if not paths and not optional:
        raise ValueError(f"No paths provided for {csv_type}.")
    return paths

def read_clients(data, client_csv_paths):
    client_ids = set()
    for path in client_csv_paths:
        if not os.path.exists(path):
            raise FileNotFoundError(f"Client CSV file not found: {path}")
        df = pd.read_csv(path)
        required_columns = ['ClientID', 'DepotID', 'Product', 'Longitude', 'Latitude']
        required_columns_2 = ['ClientID', 'Longitude', 'Latitude']
        if not all(col in df.columns for col in required_columns) and not all(col in df.columns for col in required_columns_2):
            raise ValueError(f"Client CSV missing required columns: {required_columns}")
        for index, row in df.iterrows():
            client_id = row['ClientID']
            if client_id in client_ids:
                raise ValueError(f"Duplicate ClientID found: {client_id}")
            client_ids.add(client_id)
            client_id = str(client_id.item())
            client_id = "c" + (str(int(float(client_id))) if is_float_with_zero_decimals(client_id) else client_id)
            data['N'].append(client_id)
            data['TipoNodo'][client_id] = 'Cliente'
            product = 0
            for p in ["Product-Type-A","Product-Type-B","Product-Type-C", "Product"]:
                product += row.get(p, 0)
            if product < 0:
                raise ValueError(f"Product demand must be >= 0 for ClientID {client_id}")
            product = float(product.item())
            data['DemandaEnvio'][client_id] = product
            longitude = row['Longitude']
            latitude = row['Latitude']
            if not (-180 <= longitude <= 180 and -90 <= latitude <= 90):
                raise ValueError(f"Invalid coordinates for ClientID {client_id}")
            longitude = float(longitude.item())
            latitude = float(latitude.item())
            data['UbicacionNodo'][client_id] = (latitude, longitude)

def read_vehicles(data, vehicle_csv_paths):
    vehicle_id = 1
    for path in vehicle_csv_paths:
        if not os.path.exists(path):
            raise FileNotFoundError(f"Vehicle CSV file not found: {path}")
        df = pd.read_csv(path)
        required_columns = ['VehicleType', 'Capacity', 'Range']
        if not all(col in df.columns for col in required_columns):
            raise ValueError(f"Vehicle CSV missing required columns: {required_columns}")
        for index, row in df.iterrows():
            vehicle_type_raw = row['VehicleType']
            if vehicle_type_raw == 'Gas Car':
                vehicle_type = 'Gasolina/Gas'
            elif vehicle_type_raw == 'EV':
                vehicle_type = 'EV'
            elif vehicle_type_raw == 'drone':
                vehicle_type = 'Dron'
            else:
                raise ValueError(f"Invalid VehicleType: {vehicle_type_raw}")
            capacity = row['Capacity']
            if capacity < 0:
                raise ValueError(f"Capacity must be >= 0 for VehicleID {vehicle_id}")
            range_ = row['Range']
            if range_ < 0:
                raise ValueError(f"Range must be >= 0 for VehicleID {vehicle_id}")
            vehicle_id_str = "v" + str(vehicle_id)
            data['V'].append(vehicle_id_str)
            data['TipoVehiculo'][vehicle_id_str] = vehicle_type
            data['CapacidadVehiculo'][vehicle_id_str] = capacity
            data['RangoVehiculo'][vehicle_id_str] = range_
            vehicle_id += 1

def read_depots(data, depot_csv_paths):
    depot_ids = set()
    for path in depot_csv_paths:
        if not os.path.exists(path):
            raise FileNotFoundError(f"Depot CSV file not found: {path}")
        df = pd.read_csv(path)
        required_columns = ['DepotID', 'Longitude', 'Latitude']
        if not all(col in df.columns for col in required_columns):
            raise ValueError(f"Depot CSV missing required columns: {required_columns}")
        for index, row in df.iterrows():
            depot_id = row['DepotID']
            if depot_id in depot_ids:
                raise ValueError(f"Duplicate DepotID found: {depot_id}")
            depot_ids.add(depot_id)
            depot_id = str(depot_id.item())
            depot_id = "d" + (str(int(float(depot_id))) if is_float_with_zero_decimals(depot_id) else depot_id)
            data['N'].append(depot_id)
            data['TipoNodo'][depot_id] = 'Depósito'
            longitude = row['Longitude']
            latitude = row['Latitude']
            if not (-180 <= longitude <= 180 and -90 <= latitude <= 90):
                raise ValueError(f"Invalid coordinates for DepotID {depot_id}")
            longitude = float(longitude.item())
            latitude = float(latitude.item())
            data['UbicacionNodo'][depot_id] = (latitude, longitude)

def read_depot_capacities(data, depot_capacity_csv_paths):
    for path in depot_capacity_csv_paths:
        if not os.path.exists(path):
            raise FileNotFoundError(f"Depot Capacity CSV file not found: {path}")
        df = pd.read_csv(path)
        required_columns = ['DepotID']
        if not all(col in df.columns for col in required_columns):
            raise ValueError(f"Depot Capacity CSV missing required columns: {required_columns}")
        for index, row in df.iterrows():
            depot_id = str(row['DepotID'].item())
            depot_id = "d" + (str(int(float(depot_id))) if is_float_with_zero_decimals(depot_id) else depot_id)
            if depot_id not in data['N'] or data['TipoNodo'].get(depot_id) != 'Depósito':
                raise ValueError(f"DepotID {depot_id} not found among depots.")
            product = 0
            for p in ["Product-Type-A","Product-Type-B","Product-Type-C", "Product"]:
                product += row.get(p, 0)
            if product < 0:
                raise ValueError(f"Product capacity must be >= 0 for DepotID {depot_id}")
            product = float(product.item())
            data['CapacidadDeposito'][depot_id] = product

def read_loading_products(data, loading_products_csv_paths):
    loading_cost_set = False
    unloading_cost_set = False
    loading_speed_set = False
    unloading_speed_set = False

    for path in loading_products_csv_paths:
        if not os.path.exists(path):
            raise FileNotFoundError(f"Loading Products CSV file not found: {path}")
        df = pd.read_csv(path)
        required_columns = ['Activity', 'Cost [COP/min]', 'Loading Speed [kg/min]']
        if not all(col in df.columns for col in required_columns):
            raise ValueError(f"Loading Products CSV missing required columns: {required_columns}")
        if len(df) < 1 or len(df) > 2:
            raise ValueError("Loading Products CSV must have 1 or 2 rows.")
        for index, row in df.iterrows():
            activity = row['Activity']
            cost = row['Cost [COP/min]']
            speed = row['Loading Speed [kg/min]']
            if cost < 0 or speed < 0:
                raise ValueError("Cost and Speed must be >= 0.")
            if activity == 'Loading/Unloading':
                if loading_cost_set or unloading_cost_set:
                    raise ValueError("Duplicate Loading/Unloading activity.")
                data['CostoCarga'] = cost
                data['CostoDescarga'] = cost
                data['VelocidadCarga'] = speed
                data['VelocidadDescarga'] = speed
                loading_cost_set = unloading_cost_set = True
                loading_speed_set = unloading_speed_set = True
            elif activity == 'Loading':
                if loading_cost_set:
                    raise ValueError("Duplicate Loading activity.")
                data['CostoCarga'] = cost
                data['VelocidadCarga'] = speed
                loading_cost_set = True
                loading_speed_set = True
            elif activity == 'Unloading':
                if unloading_cost_set:
                    raise ValueError("Duplicate Unloading activity.")
                data['CostoDescarga'] = cost
                data['VelocidadDescarga'] = speed
                unloading_cost_set = True
                unloading_speed_set = True
            else:
                raise ValueError(f"Invalid Activity type: {activity}")
    if not (loading_cost_set and unloading_cost_set and loading_speed_set and unloading_speed_set):
        raise ValueError("Incomplete loading/unloading data.")

def read_vehicle_types(data, vehicle_types_csv_paths):
    vehicle_types_set = set()
    for path in vehicle_types_csv_paths:
        if not os.path.exists(path):
            raise FileNotFoundError(f"Vehicle Types CSV file not found: {path}")
        df = pd.read_csv(path)
        required_columns = ['Vehicle', 'Freight Rate [COP/km]', 'Time Rate [COP/min]',
                            'Daily Maintenance [COP/day]', 'Recharge/Fuel Cost [COP/(gal or kWh)]',
                            'Recharge/Fuel Time [min/10 percent charge]', 'Avg. Speed [km/h]',
                            'Gas Efficiency [km/gal]', 'Electricity Efficency [kWh/km]']
        if not all(col in df.columns for col in required_columns):
            raise ValueError(f"Vehicle Types CSV missing required columns: {required_columns}")
        if len(df) != 3:
            raise ValueError("Vehicle Types CSV must have exactly 3 rows.")
        for index, row in df.iterrows():
            vehicle_raw = row['Vehicle']
            if vehicle_raw == 'Gas Car':
                vehicle = 'Gasolina/Gas'
            elif vehicle_raw == 'Drone':
                vehicle = 'Dron'
            elif vehicle_raw == 'Solar EV':
                vehicle = 'EV'
            else:
                raise ValueError(f"Invalid Vehicle type: {vehicle_raw}")
            if vehicle in vehicle_types_set:
                raise ValueError(f"Duplicate Vehicle type: {vehicle}")
            vehicle_types_set.add(vehicle)
            freight_rate = row['Freight Rate [COP/km]']
            time_rate = row['Time Rate [COP/min]']
            maintenance = row['Daily Maintenance [COP/day]']
            recharge_cost = row['Recharge/Fuel Cost [COP/(gal or kWh)]']
            recharge_time = row['Recharge/Fuel Time [min/10 percent charge]']
            avg_speed = row['Avg. Speed [km/h]']
            gas_efficiency = row['Gas Efficiency [km/gal]']
            electricity_efficiency = row['Electricity Efficency [kWh/km]']

            if freight_rate < 0 or time_rate < 0 or maintenance < 0:
                raise ValueError(f"Rates and maintenance must be >= 0 for Vehicle {vehicle}")
            data['TarifaFleteTipoVehiculo'][vehicle] = freight_rate
            data['TarifaHorariaTipoVehiculo'][vehicle] = time_rate
            data['MantenimientoTipoVehiculo'][vehicle] = maintenance

            if vehicle == 'EV':
                # Ignore recharge_cost and recharge_time if empty
                if pd.notnull(recharge_cost):
                    raise ValueError("Recharge/Fuel Cost should be empty for EV.")
                if pd.notnull(recharge_time):
                    raise ValueError("Recharge/Fuel Time should be empty for EV.")
                if electricity_efficiency <= 0:
                    raise ValueError("Electricity Efficiency must be > 0 for EV.")
                data['CostoRecargaTipoVehiculo'][vehicle] = 0
                data['TiempoRecargaTipoVehiculo'][vehicle] = 0
                data['EficienciaTipoVehiculo'][vehicle] = electricity_efficiency
            elif vehicle == 'Dron':
                if recharge_cost < 0 or recharge_time < 0:
                    raise ValueError(f"Recharge Cost and Time must be >= 0 for Vehicle {vehicle}")
                if electricity_efficiency <= 0:
                    raise ValueError("Electricity Efficiency must be > 0 for Dron.")
                data['CostoRecargaTipoVehiculo'][vehicle] = recharge_cost
                data['TiempoRecargaTipoVehiculo'][vehicle] = recharge_time
                data['EficienciaTipoVehiculo'][vehicle] = electricity_efficiency
            elif vehicle == 'Gasolina/Gas':
                if recharge_cost < 0 or recharge_time < 0:
                    raise ValueError(f"Fuel Cost and Time must be >= 0 for Vehicle {vehicle}")
                if gas_efficiency <= 0:
                    raise ValueError("Gas Efficiency must be > 0 for Gasolina/Gas.")
                data['CostoRecargaTipoVehiculo'][vehicle] = recharge_cost
                data['TiempoRecargaTipoVehiculo'][vehicle] = recharge_time
                data['EficienciaTipoVehiculo'][vehicle] = gas_efficiency
            else:
                raise ValueError(f"Unknown vehicle type: {vehicle}")

            if avg_speed <= 0:
                raise ValueError(f"Avg. Speed must be > 0 for Vehicle {vehicle}")
            data['VelocidadTipoVehiculo'][vehicle] = avg_speed

## Interfaz de usuario para el modelo

### Prueba del modelo

In [15]:
import time

data = {
    "N": [1, 2, 3, 4],
    "TP": ['Cliente', 'Depósito'],
    "V": [1, 2, 3],
    "TV": ['Gasolina/Gas', 'EV', 'Dron'],
    "CV": ['Terrestre', 'Aereo'],
    "CategoriaTipoVehiculo": {
        'Gasolina/Gas': 'Terrestre',
        'EV': 'Terrestre',
        'Dron': 'Aereo',
    },

    "CostoCarga": 500,
    "CostoDescarga": 300,
    "VelocidadCarga": 0.1,
    "VelocidadDescarga": 0.1,

    "TipoNodo": {
        1: 'Depósito',
        2: 'Cliente',
        3: 'Cliente',
        4: 'Cliente'
    },
    "UbicacionNodo": {
        1: (-74.153536, 4.743359),
        2: (-74.12918518424992, 4.59633980921332),
        3: (-74.02342238411416, 4.755848992747571),
        4: (-74.06261628755483, 4.737073417517297)
    },
    "DemandaEnvio": {
        1: 0,
        2: 100,
        3: 150,
        4: 200
    },
    "CapacidadDeposito": {
        1: 1000,
        2: 0,
        3: 0,
        4: 0
    },

    "TipoVehiculo": {
        1: 'Gasolina/Gas',
        2: 'EV',
        3: 'Dron'
    },
    "CapacidadVehiculo": {
        1: 500,
        2: 300,
        3: 50
    },
    "RangoVehiculo": {
        1: 500,
        2: 200,
        3: 100
    },
    "TarifaFleteTipoVehiculo": {
        'Gasolina/Gas': 1000,
        'EV': 800,
        'Dron': 1500
    },
    "TarifaHorariaTipoVehiculo": {
        'Gasolina/Gas': 200,
        'EV': 150,
        'Dron': 300
    },
    "MantenimientoTipoVehiculo": {
        'Gasolina/Gas': 5000,
        'EV': 4000,
        'Dron': 6000
    },
    "CostoRecargaTipoVehiculo": {
        'Gasolina/Gas': 8000,
        'EV': 5000,
        'Dron': 10000
    },
    "TiempoRecargaTipoVehiculo": {
        'Gasolina/Gas': 10,
        'EV': 30,
        'Dron': 20
    },
    "VelocidadTipoVehiculo": {
        'Gasolina/Gas': 60,
        'EV': 50,
        'Dron': 80
    },
    "EficienciaTipoVehiculo": {
        'Gasolina/Gas': 10,
        'EV': 5,
        'Dron': 15
    },

    "d": None
}

start_time = time.time()
# Create distance matrix
data["d"] = {cv: {i: {j: 0 for j in data["N"]} for i in data["N"]} for cv in data["CV"]}
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Data loaded successfully: {elapsed_time:.4f} seconds")

start_time = time.time()
compute_distance_matrix(data)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Distance matrix successfully calculated: {elapsed_time:.4f} seconds")

solution, solution_2 = solve_vehicle_routing_problem(data)

import json
# Custom function to handle tuples
def tuple_to_list(obj):
    if isinstance(obj, tuple):
        return list(obj)  # Convert tuples to lists
    raise TypeError(f"Type {type(obj)} not serializable")

# File name to save the JSON
output_file = "output.json"

# Write the dictionary to a JSON file
with open(output_file, 'w') as f:
    json.dump(
        solution_2,  # Data to serialize
        f,     # File object
        indent=4,  # Pretty-print with 4 spaces
        default=tuple_to_list  # Custom handler for tuples
    )

# Confirm to the user
print(f"Solution has been saved as JSON to {output_file}")

print_vehicle_routes(solution, data)

Data loaded successfully: 0.0000 seconds
https://router.project-osrm.org/table/v1/driving/4.743359,-74.153536;4.59633980921332,-74.12918518424992;4.755848992747571,-74.02342238411416;4.737073417517297,-74.06261628755483
Distance matrix successfully calculated: 0.5445 seconds
Set parameter Username
Set parameter LicenseID to value 2590111
Academic license - for non-commercial use only - expires 2025-11-25
Read LP format model from file C:\Users\cdcp2\AppData\Local\Temp\tmp7qb8n99b.pyomo.lp
Reading time = 0.01 seconds
x1: 151 rows, 87 columns, 557 nonzeros
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 4800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 151 rows, 87 columns and 557 nonzeros
Model fingerprint: 0xd3d9ca87
Variable types: 18 continuous, 69 integer (69 binary)
Coefficient statistics:
  Matrix range     [1e+00,

In [16]:
import time
data = {
    "N": [],
    "TP": ['Cliente', 'Depósito'],
    "V": [],
    "TV": ['Gasolina/Gas', 'EV', 'Dron'],
    "CV": ['Terrestre', 'Aereo'],
    "CategoriaTipoVehiculo": {
        'Gasolina/Gas': 'Terrestre',
        'EV': 'Terrestre',
        'Dron': 'Aereo',
    },

    "CostoCarga": None,
    "CostoDescarga": None,
    "VelocidadCarga": None,
    "VelocidadDescarga": None,

    "TipoNodo": {},
    "UbicacionNodo": {},
    "DemandaEnvio": {},
    "CapacidadDeposito": {},

    "TipoVehiculo": {},
    "CapacidadVehiculo": {},
    "RangoVehiculo": {},
    "TarifaFleteTipoVehiculo": {},
    "TarifaHorariaTipoVehiculo": {},
    "MantenimientoTipoVehiculo": {},
    "CostoRecargaTipoVehiculo": {},
    "TiempoRecargaTipoVehiculo": {},
    "VelocidadTipoVehiculo": {},
    "EficienciaTipoVehiculo": {},

    "d": None
}

# Option to choose base case or provide paths manually
choice = input("Choose an option:\n1. Select a base case\n2. Provide CSV paths manually\nEnter 1 or 2: ")

if choice == '1':
    base_case = select_base_case()
    paths = get_base_case_paths(base_case)
elif choice == '2':
    paths = get_manual_paths()
else:
    raise ValueError("Invalid choice. Please enter 1 or 2.")


# Configuración de las variables de entorno
group_name = 'Grupo5'
case_type = 'estandar'  # Todos los casos son 'estandar' por defecto

os.environ['GROUP_NAME'] = group_name
os.environ['CASE_TYPE'] = case_type

start_time = time.time()
# Read and process CSV files
read_clients(data, paths['clients'])
read_vehicles(data, paths['vehicles'])
read_depots(data, paths['depots'])

if paths['depot_capacities']:
    read_depot_capacities(data, paths['depot_capacities'])
else:
    # Set infinite capacity for all depots
    for depot_id in data['N']:
        if data['TipoNodo'][depot_id] == 'Depósito':
            data['CapacidadDeposito'][depot_id] = float('inf')

read_loading_products(data, paths['loading_products'])
read_vehicle_types(data, paths['vehicle_types'])

# Create distance matrix
data["d"] = {cv: {i: {j: 0 for j in data["N"]} for i in data["N"]} for cv in data["CV"]}
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Data loaded successfully: {elapsed_time:.4f} seconds")

start_time = time.time()
compute_distance_matrix(data)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Distance matrix successfully calculated: {elapsed_time:.4f} seconds")
#
# print(data)

start_time = time.time()
solution, solution_2 = solve_vehicle_routing_problem(data)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Route optimization completed: {elapsed_time:.4f} seconds")

import json
# Custom function to handle tuples
def tuple_to_list(obj):
    if isinstance(obj, tuple):
        return list(obj)  # Convert tuples to lists
    raise TypeError(f"Type {type(obj)} not serializable")

# File name to save the JSON
output_file = "output.json"

# Write the dictionary to a JSON file
with open(output_file, 'w') as f:
    json.dump(
        solution_2,  # Data to serialize
        f,     # File object
        indent=4,  # Pretty-print with 4 spaces
        default=tuple_to_list  # Custom handler for tuples
    )

# Confirm to the user
print(f"Solution has been saved as JSON to {output_file}")

print_vehicle_routes(solution, data)

Select a base case (1-4):
Data loaded successfully: 0.0364 seconds
https://router.project-osrm.org/table/v1/driving/-74.19699184741948,4.632552840424734;-74.12032773611021,4.719616298449461;-74.11027242131048,4.727691608175209;-74.06815337963502,4.712202539809157;-74.19486224157397,4.638612189685843;-74.12751619352659,4.630489482968551;-74.10178731827096,4.732421040989568;-74.0328033058973,4.703342234821396;-74.08665709611086,4.574297477559144;-74.08124218159384,4.75021190869025;-74.10993358606953,4.5363832206427785;-74.03854814565923,4.792925960208614;-74.06706883098641,4.72167778077445;-74.13826337931849,4.607707046760958;-74.12400186370824,4.650463053612691;-74.09561875464892,4.621911772492814;-74.10975623736951,4.678960680833056;-74.09547235719887,4.735973062153282;-74.10991610076434,4.550640992537941;-74.10977422186991,4.664702960902753;-74.12408925943333,4.5791740634103135
Distance matrix successfully calculated: 2.0446 seconds
Set parameter Username
Set parameter LicenseID to va

## Generación de Archivos y Reportes

In [17]:
import os
import csv

def generate_reports(solution, data):
    """
    Genera los archivos de rutas de vehículos, valor de la función objetivo e informe de costos operacionales
    a partir de la solución proporcionada.

    Parámetros:
    - solution (dict): Diccionario que contiene la solución del modelo de optimización.
                        Debe incluir las claves 'TotalCost', 'CostCarTot', 'CostDistTot',
                        'CostTpTot', 'CostRecTot', 'CostMant', 'z', 'r', 'a', 'u'.
    - data (dict): Diccionario que contiene los datos del problema, necesario para obtener 'TipoNodo' y 'TipoVehiculo'.

    Archivos Generados:
    - <GROUP_NAME>-caso-<CASE_TYPE>-<CASE_NUMBER>-ruta.csv
    - Caso<CASE_NUMBER> Objetivo.txt
    - InformeCostosOperacionales<CASE_NUMBER>.txt
    """

    # 1. Lectura de Variables de Entorno para Nombres de Archivos
    group_name = os.getenv('GROUP_NAME')
    case_type = os.getenv('CASE_TYPE')  # 'estandar' o 'especial'
    case_number = os.getenv('CASE_NUMBER')  # '1', '2', etc.

    # Validación de las variables de entorno
    if not group_name or not case_type or not case_number:
        raise ValueError("Las variables de entorno GROUP_NAME, CASE_TYPE y CASE_NUMBER deben estar definidas.")

    # Construcción de los nombres de los archivos
    routes_filename = f"{group_name}-caso-{case_type}-{case_number}-ruta.csv"
    objective_filename = f"Caso{case_number} Objetivo.txt"
    operational_cost_filename = f"InformeCostosOperacionales{case_number}.txt"

    # 2. Generación del Archivo de Rutas de los Vehículos (.csv)
    # Extracción de las variables 'r' y 'a' de la solución
    routes = solution.get('r', {})
    assignments = solution.get('a', {})
    V = data['V']
    TipoNodo = data['TipoNodo']
    TipoVehiculo = data['TipoVehiculo']

    # Mapeo de vehículos a sus depósitos asignados
    vehicle_depot = {}
    for (v, d), val in assignments.items():
        if val == 1:
            if v in vehicle_depot:
                print(f"Advertencia: El vehículo {v} tiene múltiples depósitos asignados. Solo se considerará el primero.")
                continue  # Ignorar depósitos adicionales para el mismo vehículo
            if TipoNodo[d] != 'Depósito':
                print(f"Advertencia: El nodo {d} asignado al vehículo {v} no es un depósito.")
                continue  # Ignorar si el nodo no es un depósito
            vehicle_depot[v] = d

    # Reconstrucción de las rutas para cada vehículo con depósito asignado
    routes_entries = []
    for v in V:
        if v not in vehicle_depot:
            # Vehículo sin depósito asignado, no tiene rutas que procesar
            print(f"Vehículo {v}: No hay depósito asignado. No se generará una ruta.")
            continue

        depot = vehicle_depot[v]
        current_node = depot
        steps = 0
        max_steps = len(data['N']) * len(V)  # Prevención de ciclos infinitos, ajuste según necesidad

        while True:
            # Encontrar el siguiente nodo desde el nodo actual para el vehículo v
            next_nodes = [j for (i, j, veh), val in routes.items() if i == current_node and veh == v and val > 0.5]

            if not next_nodes:
                # No hay más movimientos, fin de la ruta
                break

            if len(next_nodes) > 1:
                print(f"Vehículo {v}: Múltiples salidas desde el nodo {current_node}. Se seleccionará el primero encontrado.")
            
            next_node = next_nodes[0]  # Se asume una única transición por nodo y vehículo

            # Añadir la transición al registro
            routes_entries.append({
                'ID-Vehiculo': v,
                'ID-Origen': current_node,
                'ID-Destino': next_node
            })

            # Actualizar el nodo actual
            current_node = next_node
            steps += 1

            # Prevención de ciclos infinitos
            if steps > max_steps:
                print(f"Vehículo {v}: Ruta excede el número máximo de pasos permitidos. Posible ciclo.")
                break

            # Si se regresa al depósito, finalizar la ruta
            if current_node == depot:
                break

    # Escritura del archivo de rutas (.csv)
    with open(routes_filename, mode='w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['ID-Vehiculo', 'ID-Origen', 'ID-Destino']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for entry in routes_entries:
            writer.writerow(entry)

    # 3. Generación del Archivo de Valor de la Función Objetivo (.txt)
    total_cost = solution.get('TotalCost')
    if total_cost is None:
        raise ValueError("La solución no contiene el valor 'TotalCost'.")

    with open(objective_filename, mode='w', encoding='utf-8') as obj_file:
        obj_file.write(f"{total_cost}\n")

    # 4. Generación del Informe de Costos Operacionales (.txt)
    cost_components = ['CostCarTot', 'CostDistTot', 'CostTpTot', 'CostRecTot', 'CostMant']
    cost_map = {'CostCarTot': "Costo Carga Total", 'CostDistTot': "Costo Distancia Total", 'CostTpTot': "Costo Tarifa Horaria Total", 'CostRecTot': "Costo Recarga Total", 'CostMant': "Costo Mantenimiento"}
    missing_costs = [comp for comp in cost_components if comp not in solution]

    with open(operational_cost_filename, mode='w', encoding='utf-8') as cost_file:
        cost_file.write("Informe de Costos Operacionales\n")
        cost_file.write("================================\n\n")

        for comp in cost_components:
            cost_value = solution.get(comp)
            if cost_value is not None:
                cost_file.write(f"{cost_map[comp]}: {cost_value}\n")
            else:
                cost_file.write(f"{cost_map[comp]}: No disponible\n")

        if missing_costs:
            cost_file.write("\nAdvertencias:\n")
            for comp in missing_costs:
                cost_file.write(f"- El componente de costo '{cost_map[comp]}' no está presente en la solución.\n")

    print(f"Archivos generados exitosamente:\n- {routes_filename}\n- {objective_filename}\n- {operational_cost_filename}")

generate_reports(solution, data)

Archivos generados exitosamente:
- Grupo5-caso-estandar-4-ruta.csv
- Caso4 Objetivo.txt
- InformeCostosOperacionales4.txt


## Visualización con Mapas

In [18]:
# Import necessary libraries
import folium
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
from branca.element import Template, MacroElement

# Function to extract routes from the solution
def extract_routes(solution, data):
    routes = {}
    for v in data['V']:
        routes[v] = []
        # Find the starting depot
        depots = [d for d in data['N'] if data['TipoNodo'][d] == 'Depósito']
        assigned_depots = [d for d in depots if solution['a'].get((v, d), 0) == 1]

        if not assigned_depots:
            print(f"No depot assigned to vehicle {v}")
            continue

        depot = assigned_depots[0]
        current_node = depot
        route = [current_node]
        visited = set(route)

        while True:
            next_node = None
            for j in data['N']:
                if solution['r'].get((current_node, j, v), 0) == 1 and j not in visited:
                    next_node = j
                    break
            if next_node is None:
                # Check if route returns to depot
                if solution['r'].get((current_node, depot, v), 0) == 1:
                    route.append(depot)
                break
            else:
                route.append(next_node)
                visited.add(next_node)
                current_node = next_node

        routes[v] = route
        print(f"Vehicle {v} Route: {route}")
    return routes

# Function to create a Folium map centered on the average location of all nodes
def create_map(routes, data):
    lats = [data['UbicacionNodo'][i][0] for i in data['N']]
    longs = [data['UbicacionNodo'][i][1] for i in data['N']]
    avg_lat = sum(lats) / len(lats)
    avg_long = sum(longs) / len(longs)
    return folium.Map(location=[avg_lat, avg_long], zoom_start=12)

# Function to add a legend to the Folium map
def add_legend(m, vehicle_colors):
    # Build the legend HTML dynamically
    items = ''.join(
        f'<i style="background:{color};width:10px;height:10px;float:left;margin-right:5px;"></i>{vehicle}<br>'
        for vehicle, color in vehicle_colors.items()
    )

    legend_html = f'''
    <div style="
        position: fixed;
        bottom: 50px;
        left: 50px;
        width: 160px;
        height: auto;
        z-index:9999;
        font-size:14px;
        background-color:white;
        padding:10px;
        border:2px solid grey;
        border-radius:5px;
        ">
        <b>Vehicle Routes</b><br>
        {items}
    </div>
    '''
    legend = MacroElement()
    legend._template = Template(legend_html)
    m.get_root().add_child(legend)

# Function to plot routes and markers on the Folium map
def plot_routes_on_map(m, routes, data, solution):
    num_vehicles = len(routes)
    cmap = plt.cm.get_cmap('hsv', num_vehicles)
    vehicle_colors = {}
    vehicle_indices = {v: idx for idx, v in enumerate(routes.keys())}

    # Draw routes (polylines) for each vehicle
    for v, route in routes.items():
        if not route:
            print(f"Vehicle {v} has an empty route.")
            continue

        color = mcolors.rgb2hex(cmap(vehicle_indices[v]))
        vehicle_colors[v] = color
        route_coords = [data['UbicacionNodo'][node_id] for node_id in route if data['UbicacionNodo'].get(node_id)]

        if not route_coords:
            print(f"No coordinates found for route of vehicle {v}. Skipping.")
            continue

        folium.PolyLine(locations=route_coords, color=color, weight=5, opacity=0.5).add_to(m)

    # Add markers for all nodes
    for node_id in data['N']:
        coords = data['UbicacionNodo'].get(node_id)
        if not coords:
            continue

        lat, lon = coords
        node_type = data['TipoNodo'][node_id]

        # Determine which vehicles visit this node
        if node_type == 'Depósito':
            vehicles_visiting = set(
                v for v in data['V']
                if solution['a'].get((v, node_id), 0) == 1 or
                any(solution['r'].get((node_id, i, v), 0) == 1 or solution['r'].get((i, node_id, v), 0) == 1 for i in data['N'])
            )
            if vehicles_visiting:
                vehicle_info = "Vehicles visiting: " + ", ".join(sorted(vehicles_visiting))
                color = 'black'
            else:
                vehicle_info = "Not visited"
                color = 'gray'
        else:
            vehicle_visiting = next(
                (v for v in data['V'] if solution['z'].get((node_id, v), 0) == 1), None
            )
            if vehicle_visiting:
                color = vehicle_colors.get(vehicle_visiting, 'gray')
                vehicle_info = f"Vehicle {vehicle_visiting}"
            else:
                color = 'gray'
                vehicle_info = "Not visited"

        # Create the popup text
        popup_text = f"""
        <b>Node ID:</b> {node_id}<br>
        <b>Type:</b> {node_type}<br>
        <b>{vehicle_info}</b><br>
        """

        # Choose different icons for clients and depots
        icon = folium.Icon(
            color='blue' if node_type == 'Depósito' else 'green' if node_type == 'Cliente' else 'gray',
            icon='home' if node_type == 'Depósito' else 'user',
            prefix='fa'
        )
        folium.Marker(
            location=(lat, lon),
            icon=icon,
            popup=folium.Popup(popup_text, max_width=300),
            tooltip=f"Node {node_id} ({node_type})"
        ).add_to(m)

    # Add legend to the map
    add_legend(m, vehicle_colors)


In [19]:
# Extract routes from the solution
routes = extract_routes(solution, data)

# Create the Folium map
m = create_map(routes, data)

# Plot the routes on the map
plot_routes_on_map(m, routes, data, solution)

# Display the map inline
m

# Optionally, save the map to an HTML file
m.save('vehicle_routes.html')
print("Map has been saved to 'vehicle_routes.html'")



Vehicle v1 Route: ['d4', 'c8', 'd4']
Vehicle v2 Route: ['d12', 'c1', 'd12']
Vehicle v3 Route: ['d11', 'c4', 'c2', 'd11']
Vehicle v4 Route: ['d3', 'c5', 'c9', 'd3']
Vehicle v5 Route: ['d6', 'c6', 'c7', 'd6']
Vehicle v6 Route: ['d1', 'c3', 'd1']
Map has been saved to 'vehicle_routes.html'


C:\Users\cdcp2\AppData\Local\Temp\ipykernel_13852\543166452.py:87: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = plt.cm.get_cmap('hsv', num_vehicles)


In [20]:
from IPython.display import IFrame
display(m)